<a href="https://colab.research.google.com/github/Ashish2Parimi/Telugu_Headline_Generator/blob/main/transformer_stpic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install tensorflow_text
# !pip install sentencepiece



In [3]:
# import sentencepiece as spm
# spm.SentencePieceTrainer.train('--input=body.txt --model_prefix=body --vocab_size=8000')
# spm.SentencePieceTrainer.train('--input=body.txt --model_prefix=heading --vocab_size=2000')


In [4]:
import requests
import tensorflow_text as text
import tensorflow as tf
url = 'https://github.com/Ashish2Parimi/Telugu_Headline_Generator/raw/main/body.model'
bd_model = requests.get(url).content

In [5]:
import requests
import tensorflow_text as text
import tensorflow as tf
import numpy as np
url = 'https://github.com/Ashish2Parimi/Telugu_Headline_Generator/raw/main/heading.model'
hd_model = requests.get(url).content

In [6]:
bd_tokenizer = text.SentencepieceTokenizer(bd_model, out_type=tf.string)
hd_tokenizer = text.SentencepieceTokenizer(hd_model, out_type=tf.string)



In [7]:
import pandas as pd
import numpy as np 
from tensorflow.keras.preprocessing.sequence import pad_sequences
data = pd.read_csv('/content/train.csv')

# data = data.drop(['SNo','date','topic'],axis = 1)
data.dropna(axis = 0,inplace = True)

# data = data.applymap(lambda x:x.encode(encoding='UTF-8'))
for i,b in enumerate(data['body'].values):
    if len(b)>1000:
        data.drop(i, inplace=True)



def format_data(data):
   
    
    data = data.applymap(lambda x: str(x) if isinstance(x, int) or isinstance(x, float) else x)
    data['heading'] = data['heading'].str.replace('\d+', '')

    data['body'] = data['body'].str.replace('\d+', '')    
   
    
    return data

data = format_data(data)
data = data[:4000]
target = data.pop('heading')
data = data.pop('body')
import tensorflow as tf
dataset = tf.data.Dataset.from_tensor_slices((data.values, target.values))
len(dataset)

4000

In [8]:
val_examples = dataset.take(1000)
val_examples
train_examples = dataset.skip(1000)
train_examples 

<SkipDataset shapes: ((), ()), types: (tf.string, tf.string)>

In [9]:
for bd_examples, hd_examples in train_examples.batch(3).take(1):
  for bd in bd_examples.numpy():
    print(bd.decode('utf-8'))
    print(len(bd.decode('utf-8')))


  print()

  for hd in hd_examples.numpy():
    print(hd.decode('utf-8'))

న్యూఢిల్లీ, మే : ఏడవ వేతన సంఘం సిఫారసు చేసిన మేరకు భత్యాలను పెంచాలన్న డిమాండ్‌తో ఈ నెల న తలపెట్టిన ఒక రోజు సమ్మెను కేంద్ర ప్రభుత్వ ఉద్యోగ సంఘాలు వాయిదా వేశాయి. వేతన సంఘం సిఫారసు చేసిన మేరకు భత్యాలు పెంచేందుకు ప్రభుత్వం హామీ ఇచ్చిందని, ఈ నేపథ్యంలో సమ్మెను వాయిదా వేశామని కేంద్ర ప్రభుత్వ ఉద్యోగుల సమాఖ్య ఓ ప్రకటనలో తెలిపింది.
323
న్యూఢిల్లీ, జనవరి : అమెరికా కొత్త అధ్యక్షుడుగా బాధ్యతలు చేపట్టిన డోనాల్డ్‌ ట్రంప్‌కు ప్రధాని మోదీ అభినందనలు తెలిపారు. శుక్రవారం రాత్రి ఆయన ట్వీట్‌ చేస్తూ, అమెరికాతో ద్వైపాక్షిక సంబంధాలు మరింత పటిష్ఠం కావాలని, సహకారం ఇంకా విస్తృతం కావాలని ఆకాంక్షించారు.
252
న్యూఢిల్లీ, జూలై  (ఆంధ్రజ్యోతి): కేంద్ర పట్టణాభివృద్ధి శాఖమంత్రిగా రాజీనామ చేసిన ఎన్డీయే ఉప రాష్ట్రపతి అభ్యర్థి ఎం.వెంకయ్యనాయుడికు ఆ శాఖ ఉద్యోగులు సోమవారం ఘనంగా వీడ్కోలు పలికారు. వెంకయ్య మాట్లాడుతూ.. మూడేళ్లలో శాఖలో అందరితో అనుబంధం పెరిగిందని, అందరికీ అభినందనలు తెలిపారు.
271

న కేంద్ర ఉద్యోగుల సమ్మె లేదు
ట్రంప్‌కు మోదీ అభినందనలు 
వెంకయ్యకు ఘనంగా వీడ్కోలు


In [10]:
encoded = hd_tokenizer.tokenize(hd_examples)
encoded = hd_tokenizer.string_to_id(encoded)

for row in encoded.to_list():
  print(row)
  


[52, 174, 774, 182, 45, 232, 79, 648]
[264, 590, 83, 284, 784, 16, 124, 16, 22]
[228, 6, 30, 154, 18, 980, 48, 530, 92, 237, 22]


In [11]:
round_trip = hd_tokenizer.detokenize(encoded)
for line in round_trip.numpy():
  print(line.decode('utf-8'))

న కేంద్ర ఉద్యోగుల సమ్మె లేదు
ట్రంప్ కు మోదీ అభినందనలు
వెంకయ్యకు ఘనంగా వీడ్కోలు


In [12]:
def tokenize_pairs(bd, hd):
    bd = bd_tokenizer.tokenize(bd)
    bd = bd_tokenizer.string_to_id(bd)

    # Convert from ragged to dense, padding with zeros.
    bd = tf.dtypes.cast(bd.to_tensor(),dtype=tf.int64)

    hd = hd_tokenizer.tokenize(hd)
    hd = hd_tokenizer.string_to_id(hd)

    # Convert from ragged to dense, padding with zeros.
    hd = tf.dtypes.cast(hd.to_tensor(),dtype=tf.int64)
    return bd, hd

In [13]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [14]:
def make_batches(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))


In [15]:
train_batches = make_batches(train_examples)
val_batches = make_batches(val_examples)

In [16]:
 for (batch, (inp, tar)) in enumerate(train_batches):
   print(inp)
   print(tar)

   break

tf.Tensor(
[[  67    0    7 ...    0    0    0]
 [  62   81   52 ...    0    0    0]
 [  67    7    0 ...    0    0    0]
 ...
 [  80 1015  318 ...    0    0    0]
 [  67    0  278 ...    0    0    0]
 [3816 3518    7 ...    0    0    0]], shape=(64, 337), dtype=int64)
tf.Tensor(
[[  97  480 1332 ...    0    0    0]
 [1055   92  680 ...    0    0    0]
 [ 668    9  682 ...    0    0    0]
 ...
 [  56  810   50 ...    0    0    0]
 [ 243    8   37 ...    0    0    0]
 [  73  107   29 ...    0    0    0]], shape=(64, 25), dtype=int64)


In [17]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates
def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)


In [18]:
def create_padding_mask(seq):
  seq = tf.cast(tf.math.equal(seq, 0), tf.float32)

  # add extra dimensions to add the padding
  # to the attention logits.
  return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

In [19]:
def create_look_ahead_mask(size):
  mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
  return mask  # (seq_len, seq_len)

In [20]:
def scaled_dot_product_attention(q, k, v, mask):
  """Calculate the attention weights.
  q, k, v must have matching leading dimensions.
  k, v must have matching penultimate dimension, i.e.: seq_len_k = seq_len_v.
  The mask has different shapes depending on its type(padding or look ahead)
  but it must be broadcastable for addition.

  Args:
    q: query shape == (..., seq_len_q, depth)
    k: key shape == (..., seq_len_k, depth)
    v: value shape == (..., seq_len_v, depth_v)
    mask: Float tensor with shape broadcastable
          to (..., seq_len_q, seq_len_k). Defaults to None.

  Returns:
    output, attention_weights
  """

  matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)

  # scale matmul_qk
  dk = tf.cast(tf.shape(k)[-1], tf.float32)
  scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

  # add the mask to the scaled tensor.
  if mask is not None:
    scaled_attention_logits += (mask * -1e9)

  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)

  output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

  return output, attention_weights

In [21]:
def print_out(q, k, v):
  temp_out, temp_attn = scaled_dot_product_attention(
      q, k, v, None)
  print('Attention weights are:')
  print(temp_attn)
  print('Output is:')
  print(temp_out)

In [22]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.wq = tf.keras.layers.Dense(d_model)
    self.wk = tf.keras.layers.Dense(d_model)
    self.wv = tf.keras.layers.Dense(d_model)

    self.dense = tf.keras.layers.Dense(d_model)

  def split_heads(self, x, batch_size):
    """Split the last dimension into (num_heads, depth).
    Transpose the result such that the shape is (batch_size, num_heads, seq_len, depth)
    """
    x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(x, perm=[0, 2, 1, 3])

  def call(self, v, k, q, mask):
    batch_size = tf.shape(q)[0]

    q = self.wq(q)  # (batch_size, seq_len, d_model)
    k = self.wk(k)  # (batch_size, seq_len, d_model)
    v = self.wv(v)  # (batch_size, seq_len, d_model)

    q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
    k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
    v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

    # scaled_attention.shape == (batch_size, num_heads, seq_len_q, depth)
    # attention_weights.shape == (batch_size, num_heads, seq_len_q, seq_len_k)
    scaled_attention, attention_weights = scaled_dot_product_attention(
        q, k, v, mask)

    scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)

    concat_attention = tf.reshape(scaled_attention,
                                  (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)

    output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

    return output, attention_weights

In [23]:
def point_wise_feed_forward_network(d_model, dff):
  return tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),  # (batch_size, seq_len, dff)
      tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
  ])

In [24]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()

    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    attn_output, _ = self.mha(x, x, x, mask)  # (batch_size, input_seq_len, d_model)
    attn_output = self.dropout1(attn_output, training=training)
    out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

    ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
    ffn_output = self.dropout2(ffn_output, training=training)
    out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

    return out2

In [25]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.ffn = point_wise_feed_forward_network(d_model, dff)

    self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

    self.dropout1 = tf.keras.layers.Dropout(rate)
    self.dropout2 = tf.keras.layers.Dropout(rate)
    self.dropout3 = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):
    # enc_output.shape == (batch_size, input_seq_len, d_model)

    attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)  # (batch_size, target_seq_len, d_model)
    attn1 = self.dropout1(attn1, training=training)
    out1 = self.layernorm1(attn1 + x)

    attn2, attn_weights_block2 = self.mha2(
        enc_output, enc_output, out1, padding_mask)  # (batch_size, target_seq_len, d_model)
    attn2 = self.dropout2(attn2, training=training)
    out2 = self.layernorm2(attn2 + out1)  # (batch_size, target_seq_len, d_model)

    ffn_output = self.ffn(out2)  # (batch_size, target_seq_len, d_model)
    ffn_output = self.dropout3(ffn_output, training=training)
    out3 = self.layernorm3(ffn_output + out2)  # (batch_size, target_seq_len, d_model)

    return out3, attn_weights_block1, attn_weights_block2

In [26]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding,
                                            self.d_model)

    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]

    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, training, mask):

    seq_len = tf.shape(x)[1]

    # adding embedding and position encoding.
    x = self.embedding(x)  # (batch_size, input_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)

    return x  # (batch_size, input_seq_len, d_model)

In [27]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size,
               maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate)
                       for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(rate)

  def call(self, x, enc_output, training,
           look_ahead_mask, padding_mask):

    seq_len = tf.shape(x)[1]
    attention_weights = {}

    x = self.embedding(x)  # (batch_size, target_seq_len, d_model)
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x += self.pos_encoding[:, :seq_len, :]

    x = self.dropout(x, training=training)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training,
                                             look_ahead_mask, padding_mask)

      attention_weights[f'decoder_layer{i+1}_block1'] = block1
      attention_weights[f'decoder_layer{i+1}_block2'] = block2

    # x.shape == (batch_size, target_seq_len, d_model)
    return x, attention_weights

In [28]:
class Transformer(tf.keras.Model):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size,
               target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()

    self.tokenizer = Encoder(num_layers, d_model, num_heads, dff,
                             input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff,
                           target_vocab_size, pe_target, rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inp, tar, training, enc_padding_mask,
           look_ahead_mask, dec_padding_mask):

    enc_output = self.tokenizer(inp, training, enc_padding_mask)  # (batch_size, inp_seq_len, d_model)

    # dec_output.shape == (batch_size, tar_seq_len, d_model)
    dec_output, attention_weights = self.decoder(
        tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)  # (batch_size, tar_seq_len, target_vocab_size)

    return final_output, attention_weights

In [29]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [30]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [31]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [32]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [33]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


def accuracy_function(real, pred):
  accuracies = tf.equal(real, tf.argmax(pred, axis=2))

  mask = tf.math.logical_not(tf.math.equal(real, 0))
  accuracies = tf.math.logical_and(mask, accuracies)

  accuracies = tf.cast(accuracies, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(accuracies)/tf.reduce_sum(mask)

In [34]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')

In [35]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=bd_tokenizer.vocab_size(),
    target_vocab_size=hd_tokenizer.vocab_size(),
    pe_input=1000,
    pe_target=100,
    rate=dropout_rate)

In [36]:
def create_masks(inp, tar):
  # Encoder padding mask
  enc_padding_mask = create_padding_mask(inp)

  # Used in the 2nd attention block in the decoder.
  # This padding mask is used to mask the encoder outputs.
  dec_padding_mask = create_padding_mask(inp)

  # Used in the 1st attention block in the decoder.
  # It is used to pad and mask future tokens in the input received by
  # the decoder.
  look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
  dec_target_padding_mask = create_padding_mask(tar)
  combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)

  return enc_padding_mask, combined_mask, dec_padding_mask

In [37]:
checkpoint_path = "./checkpoints/train"

ckpt = tf.train.Checkpoint(transformer=transformer,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print('Latest checkpoint restored!!')

In [38]:
EPOCHS = 50

In [39]:
# The @tf.function trace-compiles train_step into a TF graph for faster
# execution. The function specializes to the precise shape of the argument
# tensors. To avoid re-tracing due to the variable sequence lengths or variable
# batch sizes (the last batch is smaller), use input_signature to specify
# more generic shapes.

train_step_signature = [
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
    tf.TensorSpec(shape=(None, None), dtype=tf.int64),
]


@tf.function(input_signature=train_step_signature)
def train_step(inp, tar):
  tar_inp = tar[:, :-1]
  tar_real = tar[:, 1:]

  enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

  with tf.GradientTape() as tape:
    predictions, _ = transformer(inp, tar_inp,
                                 True,
                                 enc_padding_mask,
                                 combined_mask,
                                 dec_padding_mask)
    loss = loss_function(tar_real, predictions)

  gradients = tape.gradient(loss, transformer.trainable_variables)
  optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

  train_loss(loss)
  train_accuracy(accuracy_function(tar_real, predictions))

In [40]:
import time
for epoch in range(EPOCHS):
  start = time.time()

  train_loss.reset_states()
  train_accuracy.reset_states()

  # inp -> portuguese, tar -> english
  for (batch, (inp, tar)) in enumerate(train_batches):
    train_step(inp, tar)

    if batch % 50 == 0:
      print(f'Epoch {epoch + 1} Batch {batch} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  if (epoch + 1) % 5 == 0:
    ckpt_save_path = ckpt_manager.save()
    print(f'Saving checkpoint for epoch {epoch+1} at {ckpt_save_path}')

  print(f'Epoch {epoch + 1} Loss {train_loss.result():.4f} Accuracy {train_accuracy.result():.4f}')

  print(f'Time taken for 1 epoch: {time.time() - start:.2f} secs\n')

Epoch 1 Batch 0 Loss 7.6697 Accuracy 0.0000
Epoch 1 Loss 7.6206 Accuracy 0.0007
Time taken for 1 epoch: 24.36 secs

Epoch 2 Batch 0 Loss 7.5145 Accuracy 0.0029
Epoch 2 Loss 7.4346 Accuracy 0.0077
Time taken for 1 epoch: 9.58 secs

Epoch 3 Batch 0 Loss 7.3488 Accuracy 0.0085
Epoch 3 Loss 7.2474 Accuracy 0.0147
Time taken for 1 epoch: 9.47 secs

Epoch 4 Batch 0 Loss 7.1344 Accuracy 0.0096
Epoch 4 Loss 7.0283 Accuracy 0.0190
Time taken for 1 epoch: 9.59 secs

Epoch 5 Batch 0 Loss 6.8827 Accuracy 0.0172
Saving checkpoint for epoch 5 at ./checkpoints/train/ckpt-1
Epoch 5 Loss 6.7777 Accuracy 0.0202
Time taken for 1 epoch: 9.95 secs

Epoch 6 Batch 0 Loss 6.7008 Accuracy 0.0207
Epoch 6 Loss 6.5424 Accuracy 0.0200
Time taken for 1 epoch: 9.65 secs

Epoch 7 Batch 0 Loss 6.4382 Accuracy 0.0175
Epoch 7 Loss 6.3635 Accuracy 0.0207
Time taken for 1 epoch: 9.69 secs

Epoch 8 Batch 0 Loss 6.3090 Accuracy 0.0221
Epoch 8 Loss 6.2629 Accuracy 0.0225
Time taken for 1 epoch: 9.63 secs

Epoch 9 Batch 0 Los

In [58]:
for bd_examples, hd_examples in train_examples.batch(1).take(1):
  for bd in bd_examples.numpy():
    print(bd.decode('utf-8'))
    sentence = bd.decode('utf-8')
  print()

  for hd in hd_examples.numpy():
    print(hd.decode('utf-8'))
    ground_truth = hd.decode('utf-8')

న్యూఢిల్లీ, మే : ఏడవ వేతన సంఘం సిఫారసు చేసిన మేరకు భత్యాలను పెంచాలన్న డిమాండ్‌తో ఈ నెల న తలపెట్టిన ఒక రోజు సమ్మెను కేంద్ర ప్రభుత్వ ఉద్యోగ సంఘాలు వాయిదా వేశాయి. వేతన సంఘం సిఫారసు చేసిన మేరకు భత్యాలు పెంచేందుకు ప్రభుత్వం హామీ ఇచ్చిందని, ఈ నేపథ్యంలో సమ్మెను వాయిదా వేశామని కేంద్ర ప్రభుత్వ ఉద్యోగుల సమాఖ్య ఓ ప్రకటనలో తెలిపింది.

న కేంద్ర ఉద్యోగుల సమ్మె లేదు


In [59]:
sentence = tf.convert_to_tensor([sentence])
sentence = bd_tokenizer.tokenize(sentence)
sentence = bd_tokenizer.string_to_id(sentence)
sentence = tf.dtypes.cast(sentence.to_tensor(),dtype=tf.int64)
encoder_input = sentence

In [64]:
start= hd_tokenizer.tokenize([''])[0]
output = tf.convert_to_tensor([start])
output = tf.expand_dims(output, 0)

In [65]:

for i in range(40):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
          encoder_input, output)
    predictions, attention_weights = transformer(encoder_input,
                                                output,

                                                False,
                                                enc_padding_mask,
                                                combined_mask,
                                                dec_padding_mask)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.argmax(predictions, axis=-1)

    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

    # return the result if the predicted_id is equal to the end token


TypeError: ignored

In [50]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence}')
  # print(f'{"Prediction":15s}: {tokens.numpy().decode("utf-8")}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [53]:
print_translation(sentence, ground_truth)


Input:         : న్యూఢిల్లీ, మే : ఏడవ వేతన సంఘం సిఫారసు చేసిన మేరకు భత్యాలను పెంచాలన్న డిమాండ్‌తో ఈ నెల న తలపెట్టిన ఒక రోజు సమ్మెను కేంద్ర ప్రభుత్వ ఉద్యోగ సంఘాలు వాయిదా వేశాయి. వేతన సంఘం సిఫారసు చేసిన మేరకు భత్యాలు పెంచేందుకు ప్రభుత్వం హామీ ఇచ్చిందని, ఈ నేపథ్యంలో సమ్మెను వాయిదా వేశామని కేంద్ర ప్రభుత్వ ఉద్యోగుల సమాఖ్య ఓ ప్రకటనలో తెలిపింది.
Ground truth   : న కేంద్ర ఉద్యోగుల సమ్మె లేదు


In [49]:
def evaluate(sentence, max_length=40):
  # inp sentence is portuguese, hence adding the start and end token
  sentence = tf.convert_to_tensor([sentence])
  sentence = bd_tokenizer.tokenize(sentence)
  sentence = bd_tokenizer.string_to_id(sentence)
  sentence = tf.dtypes.cast(sentence.to_tensor(),dtype=tf.int64)
  encoder_input = sentence

  # as the target is english, the first word to the transformer should be the
  # english start token.
  start= hd_tokenizer.tokenize([''])[0]
  output = tf.convert_to_tensor([start])
  output = tf.expand_dims(output, 0)

  for i in range(max_length):
    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(
        encoder_input, output)

    # predictions.shape == (batch_size, seq_len, vocab_size)
    predictions, attention_weights = transformer(encoder_input,
                                                 output,
                                                 False,
                                                 enc_padding_mask,
                                                 combined_mask,
                                                 dec_padding_mask)

    # select the last word from the seq_len dimension
    predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)

    predicted_id = tf.argmax(predictions, axis=-1)

    # concatentate the predicted_id to the output which is given to the decoder
    # as its input.
    output = tf.concat([output, predicted_id], axis=-1)

    # return the result if the predicted_id is equal to the end token
    if predicted_id == end:
      break

  # output.shape (1, tokens)
  text = hd_tokenizer.detokenize(output)  # shape: ()


  return text, attention_weights

In [55]:
translated_text, attention_weights = evaluate(sentence)


TypeError: ignored